In [1]:

from qiskit import QuantumCircuit, transpile
from sympy import Matrix, init_printing
from qiskit.quantum_info import Operator
init_printing()
def op(qc):
    return Matrix(Operator(qc).data.round(3))

n_max = 30


No Ancilla

In [2]:
# for i in range(2, n_max):
#     qc = QuantumCircuit(i)
    
#     qc.mcx(list(range(1, i)), 0)
    
#     qc1 = transpile(qc, basis_gates=["h", "p", "u3", "cx"])
#     print(i, qc1.depth(), qc1.depth(lambda x: x[0].name == "cx"))


1 ancilla (default)

In [3]:
for i in range(2, n_max):
    qc = QuantumCircuit(i+1)
    
    qc.mcx(list(range(1, i)), i, ancilla_qubits=[0], mode="recursion")
    
    qc1 = transpile(qc, basis_gates=["h", "p", "u3", "cx"])
    print(i, qc1.depth(), qc1.depth(lambda x: x[0].name == "cx"))


2 1 1
3 11 6
4 27 14
5 65 35
6 87 45
7 155 84
8 225 121
9 253 135
10 316 165
11 444 238
12 571 310
13 731 396
14 833 449
15 889 477
16 942 505
17 1059 559
18 1142 598
19 1402 746
20 1654 890
21 1908 1034
22 2102 1144
23 2422 1316
24 2694 1461
25 2884 1561
26 3018 1630
27 3130 1686
28 3271 1756
29 3377 1812


With 1 ancilla (modified)

Based on [Efficient constructions for simulating Multi
Controlled Quantum Gates](https://www.iccs-meeting.org/archive/iccs2022/papers/133530169.pdf) and Qiskit's [MCXRecursive](https://qiskit.org/documentation/_modules/qiskit/circuit/library/standard_gates/x.html#MCXRecursive)

In [14]:
from functools import cache
import numpy as np

def mcx(qc: QuantumCircuit, target, ctrls=None, ancilla=None):
    if ctrls is None or len(ctrls) == 0:
        return qc.x(target)
    if len(ctrls) < 5:
        return qc.mcx(ctrls, target)

    # recurse
    middle = (1 + len(ctrls)) // 2
    first_half, second_half = ctrls[:middle], [*ctrls[middle:], ancilla]

    qc.mcx(first_half, ancilla, second_half[:-1], mode="v-chain-dirty")
    qc.mcx(second_half, target, first_half, mode="v-chain-dirty")
    qc.mcx(first_half, ancilla, second_half[:-1], mode="v-chain-dirty")
    qc.mcx(second_half, target, first_half, mode="v-chain-dirty")
    
@cache
def depth_mcx(i: int) -> tuple[int, int]:
    if i < 1:
        return 0, 0
    
    qc = QuantumCircuit(i+1)
    mcx(qc, 0, list(range(1, i)), i)
    
    # optimization_level=3 doesn't do anything
    qc = transpile(qc, basis_gates=["u", "cx"])
    
    return qc.depth(), sum(qc.count_ops().values()) # qc.depth(lambda x: x[0].name == "cx")
    
for i in range(n_max):
    print(i, *depth_mcx(i))


0 0 0
1 1 1
2 1 1
3 11 14
4 27 30
5 65 77
6 104 119
7 151 171
8 188 223
9 241 275
10 278 327
11 337 379
12 374 431
13 433 483
14 470 535
15 529 587
16 566 639
17 625 691
18 662 743
19 721 795
20 758 847
21 817 899
22 854 951
23 913 1003
24 950 1055
25 1009 1107
26 1046 1159
27 1105 1211
28 1142 1263
29 1201 1315


Depth of Shift operation

In [5]:
def shift(qc: QuantumCircuit, k: int = 1, targets=None, control=None, ancilla=None) -> None:
    if k == 0:
        return
    if targets is None:
        targets = qc.qubits

    # Increment / Decrement for
    for _ in range(abs(k)):
        for i in range(len(targets))[:: -np.sign(k)]:
            controls = list(targets[:i])

            if control is not None:
                controls += [control]

                mcx(qc, targets[i], ctrls=controls, ancilla=ancilla)

@cache
def depth_controlled_shift(i: int):
    if i < 2:
        return ValueError(f"number of qubits must be at least 2, got {i}")
    
    qc = QuantumCircuit(i+1)
    shift(qc, targets=list(range(i-1)), control=i-1, ancilla=i)
    
    # optimization_level=3 doesn't do anything
    qc = transpile(qc, basis_gates=["u", "cx"])
    
    return qc.depth(), sum(qc.count_ops().values())

# for i in range(2, n_max):
#     print(i, *depth_controlled_shift(i))


In [6]:
import numpy as np
from qiskit import QuantumCircuit, transpile

rng = np.random.default_rng(seed=42069)

def c2q_real_depth(n, f=0):
    qc = QuantumCircuit(n+f)
    
    targets = qc.qubits[:n]
    for i, t in enumerate(targets[::-1]):
        j = len(targets) - i
        
        ctrls = qc.qubits[j:]
        
        params = 2*np.pi*rng.random(2**len(ctrls))
        qc.ucry(list(params), ctrls, t)
        
    qc = transpile(qc, basis_gates=["ry", "cx"], optimization_level=3)
    
    # return qc.draw('mpl', reverse_bits=True)
    return qc.depth(), sum(qc.count_ops().values()) # qc.depth(lambda x: x[0].name == "cx")
    
# c2q_real_depth(3, 2)


Depth of Convolution operation

In [7]:
# def depth_convolution(n: list[int], n_f: list[int], features: int = 0):
    
#     ### FIND MAX DIMENSION WITH MAX SIZE FILTER
#     n_max, n_f_max = None, None
#     for n_i, n_f_i in zip(n, n_f):
#         # initial case
#         if n_max is None or n_f_max is None:
#             n_max, n_f_max = n_i, n_f_i
            
#         if n_i > n_max:
#             n_max, n_f_max = n_i, n_f_i
        
#         if (n_i == n_max) and (n_f_i > n_f_max):
#             n_max, n_f_max = n_i, n_f_i
    
#     total_depth, gate_count = zip(*(depth_controlled_shift(n_max + 1 - j) for j in range(n_f_max)))
        
#     total_depth = sum(total_depth) + 1 # H gate
#     gate_count = sum(gate_count) + 3 # SWAP permutations
    
#     # U_F depth (FROM TC2022):
#     c2q_depth = c2q_real_depth(sum(n_f), features)
#     total_depth += c2q_depth[0]
#     gate_count += c2q_depth[1]
    
#     # print(c2q_depth)
    
#     # N_f = 2**sum(n_f)
#     # total_depth += 3*N_f - sum(n_f) - 4
#     # cnot_depth += N_f - 2
    
#     return total_depth, gate_count


In [8]:
# for i in range(1, n_max):
#     print(i, *depth_convolution([i, i], [1, 1], 2))


In [9]:
# for j in range(1, 10):
#     print(j, *depth_convolution([4, 4], [j, j], 1))


MQCC

In [10]:
dims = [16, 16]
kernel = [2, 2]
num_features = 1
num_classes = 2
num_layers = 4


In [11]:
import numpy as np
from qiskit import QuantumCircuit
from qcc.quantum import to_qubits
from qcc.quantum.qubits import Qubits

rng = np.random.default_rng(seed=42069)

def mac(qc: QuantumCircuit, mac_qubits: list, feature_qubits: list):
    qubits = mac_qubits + feature_qubits
    for i, t in enumerate(mac_qubits):
        ctrls = qubits[i+1:]
        
        params = 2*np.pi*rng.random(2**len(ctrls))
        qc.ucry(list(params), ctrls, t)
    

def convolution(qc: QuantumCircuit, data_qubits: Qubits, kernel_qubits: Qubits, feature_qubits: Qubits, ancilla):
    for i, (dq, fq) in enumerate(zip(data_qubits, kernel_qubits)):
        for i, control_qubit in enumerate(fq):
            shift(qc, -1, targets=dq[i:], control=control_qubit, ancilla=ancilla)
            
    mac(qc, kernel_qubits.flatten(), feature_qubits.flatten())


In [12]:
from qiskit import transpile

def mqcc(dims: list[int], kernel: list[int], num_features: int = 1, num_classes: int = 2, num_layers: int = 1):

    dims_q = [to_qubits(x) for x in dims]
    kernel_q = [to_qubits(x) for x in kernel]
    features_q = to_qubits(num_features)
    classes_q = to_qubits(num_classes)

    qc = QuantumCircuit(sum(dims_q) + (num_layers * sum(kernel_q)) + features_q + classes_q + 1)

    top = np.cumsum(dims_q)
    bottom = [t - d for d, t in zip(dims_q, top)]
    data_qubits = Qubits([qc.qubits[b:t] for b, t, in zip(bottom, top)])
    
    kernel_qubits, n = Qubits(), data_qubits.total
    for _ in range(num_layers):
        for fq in kernel_q:
            kernel_qubits += [qc.qubits[n : n + fq]]
            n += fq

    n = data_qubits.total + kernel_qubits.total
    feature_qubits = Qubits(qc.qubits[n:n + features_q])

    n += features_q
    class_qubits = Qubits(qc.qubits[n : n + classes_q])
    
    ancilla = qc.qubits[-1]

    qc.h(Qubits(kernel_qubits + feature_qubits + class_qubits).flatten())
    for i in range(num_layers):
        kernel_qubits_i = Qubits(kernel_qubits[i*sum(kernel_q): (i+1)*sum(kernel_q)])
        convolution(qc, data_qubits, kernel_qubits_i, feature_qubits, ancilla)
        
        for j, fq in enumerate(kernel_q):
            data_qubits[j] = data_qubits[j][fq:]
        
        # qc.barrier()
        
    mac(qc, data_qubits.flatten() + feature_qubits.flatten(), class_qubits.flatten())

    return qc

qc = mqcc(
    dims=dims,
    kernel=kernel,
    num_features=num_features,
    num_classes=num_classes,
    num_layers=num_layers,
)
# display(qc.draw('mpl', reverse_bits=True))

qc = transpile(qc, basis_gates=["u", "cx"], optimization_level=3)
qc.depth(), sum(qc.count_ops().values())


/Users/mingyoungjeng/Documents/Spring 2023.nosync/quantum-convolutional-classifier/.env/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/mingyoungjeng/Documents/Spring 2023.nosync/quantum-convolutional-classifier/.env/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [13]:
from qiskit import transpile

def mqcc_optimized(dims: list[int], kernel: list[int], num_features: int = 1, num_classes: int = 2, num_layers: int = 1):

    dims_q = [to_qubits(x) for x in dims]
    kernel_q = [to_qubits(x) for x in kernel]
    features_q = to_qubits(num_features)
    classes_q = to_qubits(num_classes)

    qc = QuantumCircuit(sum(dims_q) + features_q + classes_q + 1)

    top = np.cumsum(dims_q)
    bottom = [t - d for d, t in zip(dims_q, top)]
    data_qubits = Qubits([qc.qubits[b:t] for b, t, in zip(bottom, top)])

    n = data_qubits.total
    feature_qubits = Qubits(qc.qubits[n:n + features_q])

    n += features_q
    class_qubits = Qubits(qc.qubits[n : n + classes_q])
    ancilla = qc.qubits[-1]

    qc.h(Qubits(kernel_qubits + feature_qubits + class_qubits).flatten())
    for i in range(num_layers):
        kernel_qubits = Qubits()
        for j, fq in enumerate(kernel_q):
            kernel_qubits += [data_qubits[j][:fq]]
            data_qubits[j] = data_qubits[j][fq:]

        convolution(qc, data_qubits, kernel_qubits, feature_qubits, ancilla)
        
        # qc.barrier()
        
    mac(qc, data_qubits.flatten() + feature_qubits.flatten(), class_qubits.flatten())

    return qc

qc = mqcc_optimized(
    dims=dims,
    kernel=kernel,
    num_features=num_features,
    num_classes=num_classes,
    num_layers=num_layers,
)
# display(qc.draw('mpl', reverse_bits=True))

qc = transpile(qc, basis_gates=["u", "cx"], optimization_level=3)
qc.depth(), sum(qc.count_ops().values())


UnboundLocalError: cannot access local variable 'kernel_qubits' where it is not associated with a value